## 1. Import Dependencies

In [34]:
import gymnasium as gym 
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os
import cv2

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## 2. Types of Spaces

In [35]:
Discrete(3).sample()

1

In [36]:
Box(0,1,shape=(3,3)).sample()

array([[0.04882846, 0.16195913, 0.67356384],
       [0.03744353, 0.94374436, 0.8071024 ],
       [0.79365724, 0.65500504, 0.5001494 ]], dtype=float32)

In [37]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(2, array([0.6485466 , 0.66023684, 0.23392445], dtype=float32))

In [38]:
Dict({'height':Discrete(2), 'speed':Box(0,100,shape=(1,)), 'color':MultiBinary(4)}).sample()

OrderedDict([('color', array([1, 0, 0, 1], dtype=int8)),
             ('height', 1),
             ('speed', array([71.890045], dtype=float32))])

In [39]:
MultiBinary(4).sample()

array([0, 1, 1, 1], dtype=int8)

In [40]:
MultiDiscrete([5,2,2]).sample()

array([3, 1, 1], dtype=int64)

In [41]:
# 4 key types of spaces - Box, Discrete, MultiBinary and MultiDiscrete
# Two wrapper spaces - Tuple and Dict - Help group different spaces together

## 3. Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degrees

In [42]:
Box(low=np.array([0]), high = np.array([100])).sample()

array([79.194374], dtype=float32)

In [43]:
Box(low = 0, high = 100, shape = (1,)).sample()

array([25.6841], dtype=float32)

In [44]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)  # Actions: -1, 0, +1
        self.observation_space = Box(low=np.array([0], dtype=np.float32), 
                                     high=np.array([100], dtype=np.float32))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60

    def step(self, action):
        self.state += action - 1
        self.shower_length -= 1
        reward = 1 if 37 <= self.state <= 39 else -1
        terminated = False  # No true termination condition
        truncated = self.shower_length <= 0  # Ends when time is up
        return np.array([self.state], dtype=np.float32), reward, terminated, truncated, {}

    def render(self):
        pass  # Implement visualization if needed

    def reset(self, seed=None):
        super().reset(seed=seed)  # Ensure proper Gym compliance
        self.state = np.array([38 + random.randint(-3, 3)], dtype=np.float32)
        self.shower_length = 60
        return self.state, {}  # Return observation and info

In [45]:
env = ShowerEnv()

In [46]:
env.observation_space.sample()

array([35.311485], dtype=float32)

In [47]:
env.action_space.sample()

0

In [48]:
env.reset()

(array([35.], dtype=float32), {})

## 4. Test Environment

In [49]:
episodes = 5
episode_scores = []

for episode in range(1, episodes + 1):
    obs, _ = env.reset()  # Reset env
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()  # Random action
        obs, reward, terminated, truncated, info = env.step(action)  # ✅ Correct unpacking
        score += reward
        done = terminated or truncated  # ✅ Check both conditions

    episode_scores.append(score)  # Store the score
    print(f"Episode {episode}: Score = {score:.2f}")    

env.close()


Episode 1: Score = -12.00
Episode 2: Score = -48.00
Episode 3: Score = 12.00
Episode 4: Score = -32.00
Episode 5: Score = -42.00


## 5. Train Model

In [50]:
log_path = os.path.join("Training","Logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path, device="cpu")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [51]:
model.learn(total_timesteps=200000)

Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -36.6    |
| time/              |          |
|    fps             | 1834     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -32.5       |
| time/                   |             |
|    fps                  | 1118        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009574827 |
|    clip_fraction        | 0.0494      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -6.37e-05   |

## 6. Save Model

In [72]:
shower_path = os.path.join('Training','Saved Models','Shower_40K_PPO')

In [53]:
#shower_path = os.path.join('Training','Saved Models','Shower_200K_PPO')

In [54]:
model.save(shower_path)

In [73]:
del model

In [74]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## 7. Evaluate

In [75]:
evaluate_policy(model, env, n_eval_episodes=10)

(59.0, 1.0)